## Tech Challenge - Fase 2


####Dicionário das features da base de dados
*   **data:** data da amostragem
*   **ultimo:** valor final em que fechou a bolsa
*   **abertura:** valor inicial quando abriu a bolsa
*   **maxima:** valor máximo alcançado naquele dia
*   **minima:** valor mínimo alcançado naquele dia
*   **volume:** a quantidade total de de ativos financeiros que foram negociados durante um determinado período, neste caso, durante um dia
*   **variacao:** percentual de variacao do ultimo do dia anterior comparado ao ultimo do dia atual. (%) Calcula-se com o mais recente/menos recente. Exemplo, dividindo 133,533/132,753 temos 1,0059... multiplica-se por 100 para capturar a porcentagem somente após o número 1.Ou seja, aumentou 0,59%(número positivo por causa do 1 na frente, se fosse com o 0 na frente, entao teríamos que colocar um sinal de negativo e também subtrair o valor encontrado de 1)

### Obtendo os dados

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df =  pd.read_csv("/content/drive/MyDrive/Tech Challenges/Fase 2/Base dados/8 anos com pandemia/Dados Históricos - Ibovespa.csv", sep=',');
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,28.12.2023,134.185,134.194,134.389,133.832,"7,81M","-0,01%"
1,27.12.2023,134.194,133.523,134.195,133.328,"6,17M","0,49%"
2,26.12.2023,133.533,132.753,133.645,132.753,"5,15M","0,59%"
3,22.12.2023,132.753,132.182,133.035,132.094,"8,23M","0,43%"
4,21.12.2023,132.182,130.826,132.277,130.822,"8,68M","1,05%"


###Análise Exploratória e Tratamento de Dados

In [12]:
df.rename(columns={"Data":"data", "Último":"ultimo", "Abertura":"abertura", "Máxima":"maxima", "Mínima":"minima", "Vol.": "volume", "Var%":"variacao" }, inplace=True)

In [13]:
df.shape

(1982, 7)

In [14]:
df.isnull().sum()

data        0
ultimo      0
abertura    0
maxima      0
minima      0
volume      1
variacao    0
dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.isnull().sum()

data        0
ultimo      0
abertura    0
maxima      0
minima      0
volume      0
variacao    0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1981 entries, 0 to 1981
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1981 non-null   object 
 1   ultimo    1981 non-null   float64
 2   abertura  1981 non-null   float64
 3   maxima    1981 non-null   float64
 4   minima    1981 non-null   float64
 5   volume    1981 non-null   object 
 6   variacao  1981 non-null   object 
dtypes: float64(4), object(3)
memory usage: 123.8+ KB


####Transformando a coluna 'data' para DateTime para facilitar as análises


In [18]:
df['data'] = pd.to_datetime(df['data'])
df.head(3)

<ipython-input-18-e523faec7500>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data'] = pd.to_datetime(df['data'])


,data,ultimo,abertura,maxima,minima,volume,variacao
0,2023-12-28,134.185,134.194,134.389,133.832,"7,81M","-0,01%"
1,2023-12-27,134.194,133.523,134.195,133.328,"6,17M","0,49%"
2,2023-12-26,133.533,132.753,133.645,132.753,"5,15M","0,59%"
